# Finding open security groups with the AWS API

AWS API Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

In this exercise we'll start digging into the AWS API and learn how to find security groups that are open to the world and the EC2 instances that use them.

Steps:
1) Find all open security groups and output them in an easy-to-use format
2) Find of the security groups used by running instances
3) Check a whitelist to see whether each instance should be allowed to use an open security group
4) Disable non-whitelisted security groups 
5) Send a notification to be logged in Splunk

In [27]:
# First we'll import boto3 to be used for everything else
import boto3

## describe_security_groups
First we'll take a look at what visibility AWS offers us into security group configurations. The `describe_security_group` endpoint can return information for all security groups, giving us a good starting point.

Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.describe_security_groups

In [75]:
def open_security_groups():
    """Return all security groups that are allow inbound connections from the """
    ec2_client = boto3.client("ec2")
    security_groups = ec2_client.describe_security_groups(
        Filters=[
            {"Name": "ip-permission.cidr", "Values": ["0.0.0.0/0"]},
        ]
    )
    groups_whitelist = ["allow_splunk_ports_ingress"]
    ports_whitelist = [22, 80, 443, 8080, 8000]
    open_groups = {}
    for sg in security_groups["SecurityGroups"]:
        for permission in sg["IpPermissions"]:
            if sg["GroupName"] not in groups_whitelist and permission["ToPort"] not in ports_whitelist:
                open_groups[sg["GroupId"]] = sg["GroupName"]
    return open_groups

open_security_groups()

{'sg-071946f055fc746a9': 'Malicious security group',
 'sg-07fbb9a5a95041c4e': 'more evil security groups',
 'sg-08313b169ba1edf2e': 'despicable security group'}

In [76]:
def instance_security_groups():
    """Return a summary of all the security group IDs assocated with running instances.
    
    This format will look like: 
        {'i-096e3b9655241f365': ['sg-05777ecea90c47aae'], ...}
    """
    ec2_client = boto3.client("ec2")
    running_instances = ec2_client.describe_instances(
        Filters=[
            {"Name": "instance-state-name", "Values": ["running", "stopped"]},
        ]
    )
    instances = {}
    for reservation in running_instances["Reservations"]:
        for instance in reservation["Instances"]:
            instance_id = instance["InstanceId"]
            for iface in instance["NetworkInterfaces"]:
                instances[instance_id] = []
                for group in iface["Groups"]:
                    instances[instance_id].append(group["GroupId"])
    return instances
    
instance_security_groups()

{'i-096e3b9655241f365': ['sg-071946f055fc746a9',
  'sg-08313b169ba1edf2e',
  'sg-07fbb9a5a95041c4e',
  'sg-05777ecea90c47aae']}

Now that we have easier-to-parse lists of all security groups and the security groups our instances use, we can go through them and easily find which instances are using open security groups.

In [77]:
open_groups = open_security_groups().keys()
instance_groups = instance_security_groups()
for instance, groups in instance_groups.items():
    instance_open_groups = list(set(groups).intersection(open_groups))
    if instance_open_groups:
        print(instance, instance_open_groups)

i-096e3b9655241f365 ['sg-08313b169ba1edf2e', 'sg-071946f055fc746a9', 'sg-07fbb9a5a95041c4e']


Now that we know we can identify instances with open groups, let's write the code to remove that open group

In [78]:
def remove_security_group(instance_id, sg_id):
    ec2 = boto3.client('ec2')
    default_group_id = ec2.describe_security_groups(
        Filters=[
            dict(Name='group-name', Values=['default'])
        ]
    )['SecurityGroups'][0]['GroupId']
    this_group_name = ec2.describe_security_groups(
        Filters=[
            dict(Name='group-id', Values=[sg_id])
        ]
    )['SecurityGroups'][0]['GroupName']    
    ec2_resource = boto3.resource("ec2")
    # Here we check if the word "despicable" is in the security group and refuse to delete if not.
    # We don't want accidentally delete something!
    if not "despicable" in this_group_name:
        return f"Cowardly refusing to delete non-despicable group {this_group_name} ({sg_id})"
    instance = ec2_resource.Instance(instance_id)
    new_groups = [g["GroupId"] for g in instance.security_groups if g["GroupId"] != sg_id]
    # Security groups can't be empty, so if this list is empty use the default security group
    if not new_groups:
        new_groups = [default_group_id]
    instance.modify_attribute(Groups=new_groups)
    return "Done"

Now we can put it all together:

In [79]:
def remediate_open_security_groups():
    open_groups = open_security_groups().keys()
    instance_groups = instance_security_groups()
    removal_summary = []
    for instance, groups in instance_groups.items():
        instance_open_groups = list(set(groups).intersection(open_groups))
        for group in instance_open_groups:
            removal_summary.append(f"Removing {group} from {instance}: {remove_security_group(instance, group)}")
    return removal_summary

remediate_open_security_groups()

['Removing sg-08313b169ba1edf2e from i-096e3b9655241f365: Done',
 'Removing sg-071946f055fc746a9 from i-096e3b9655241f365: Cowardly refusing to delete non-despicable group Malicious security group (sg-071946f055fc746a9)',
 'Removing sg-07fbb9a5a95041c4e from i-096e3b9655241f365: Cowardly refusing to delete non-despicable group more evil security groups (sg-07fbb9a5a95041c4e)']